# Quiz - Managing Conversation History
- trim_messages()
- RunnablePassthrough
- itemgetter

session_id 지정하여 대화 진행.
llm 모델이 과거 대화를 알지 못하는 상황 만들기.

In [3]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import SystemMessage, HumanMessage, trim_messages
from operator import itemgetter
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as f:
    law = f.read()

llm = ChatOpenAI(
    model = 'gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    )

template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
        아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
        답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
        - 문서: {law}'''),
        MessagesPlaceholder(variable_name="messages"),
        ('user', '{query}')
    ]

prompt = ChatPromptTemplate.from_messages(template)

trimmer = trim_messages(
    max_tokens=5_000,
    strategy="last",
    token_counter=llm,
    include_system=True, #Systemmessage 포함 출력
    allow_partial=False,
    start_on="Human",
)

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    |prompt
    |llm
)

# 세션별 대화 이력을 임시 메모리 저장
store = {}


config = {"configurable":{"session_id":"asdf"} }


def get_session_history(session_id: str) -> BaseChatMessageHistory: #부모클래스
	if session_id not in store:
		store[session_id] = InMemoryChatMessageHistory() #자식클래스
	return store[session_id]


with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="query",
    history_messages_key="messages"
)



while True:
    query = input('질문을 입력하시오. 관두려면 S')

    if query.upper() == 'S':
        break

    response = with_message_history.invoke(
        {"query": query, "law": law},
        config=config,
    )

    print('\n' + '=' * 50 + '\n')  # 이 줄이 동일한 블록 안으로 들어가야 함



KeyboardInterrupt



#### 질문
- 육아휴직 기간?
- 조건?
- 대상자?
- 수당?
- 첫 번째 질문이 뭐야?
- 질문 정리해줘.
- 네 역할은?
- 육아휴직 질문에 대한 답변 정리.

In [5]:
# 이전 대화기록 전부 출력하기
for messages in get_session_history("asdf").messages:
	print(f'[{messages.type.upper()}]: {messages.content}')
	print()

In [6]:
# trimmer 사용한 후 남은 대화 기록 
response.content

NameError: name 'response' is not defined

In [8]:
store

{'asdf': InMemoryChatMessageHistory(messages=[])}